# Lab 4: Laboratory Notes - Week 4: Wrangling Data

We will introduce fewer laboratory materials for the next two weeks (week 4 and Week 5). This is so that:

1) you will have time to review and explore earlier materials.  Do remember that mastering programming requires you need to understand how and why the code works, and the best way is to try different permutations to figure things out,  
2) you get more experience in working with the Jupyter Notebook environment, understanding how the values of the DataFrame or variables changes through the sequence of execution, and explore the Markdown (and for some, HTML and even maybe LaTex!), and  
3) you have the opportunity to start on your coursework 1,  which is due in Week 7.

We will now briefly look at data issues, and how we can treat them.

## Data Wrangling in Python

You should have a file called "<span style="color:red">patients.txt</span>" on Canvas, which is sample in a fixed width (column) format file.  As it is a plain text file, you can open this using any text editor (e.g., Notepad). The first 4 lines of the file Patients.txt contents look like

001M11/11/1998 88140 80  10  
016F11/13/1998 84120 78  X0  
033X10/21/1998 68190100  31  
004F01/01/1999101200120  5A  

Save this file to the same folder location as your Jupyter Notebook and then start a new Python 3 Notebook.

The data in "<span style="color:red">patients.txt</span>" is not formatted as a CSV file, but rather in a fixed-width format, so we'll try to load the data into a DataFrame by calling the Pandas <span style="color:red">read_fwf()</span> function (fwf stands for fixed width format) rather than the <span style="color:red">read_csv()</span> function.

<span style="color:red">import pandas as pd  
df = pd.read_fwf('patients.txt', colspecs='infer', header=None)</span>

Note that we have asked the Pandas library to try to work out what columns are in the data using the parameter <span style="color:red">colspecs="infer"</span>. We have also told it that there are no column headers on the first line of the file. Did it work? Print the DataFrame to see:

<span style="color:red">print(df)</span>

How many columns did Pandas find? To see the dimension of the table found you can also type:

<span style="color:red">df.shape</span>

You should get (35, 2) which means that it has 30 rows and 2 columns.  From our manual inspection, we probably can tell that this is not what we want.  Looking at a line of the raw content, it seems that the first 3 digits means something, followed by an indication that the person is either a Male (M) or Female (F), then followed by a date and so on. 

Let’s assume that we are aware that the data should contain the following information:

* A 3-digit identity
* Gender
* Date of visit
* 3-digit heart rate reading
* 3-digit Systolic blood pressure
* 3-digit Diastolic blood pressure
* 3-digit Diagnostic code
* 1 digit to indicate whether it is an Adverse Event

## Reading the File

Since Pandas can’t do it automatically (the <span style="color:red">colspecs="infer"</span>), we will help Pandas to load the data by specifying the width of each column in the original data:

<span style="color:red">df = pd.read_fwf('patients.txt', widths=[3,1,10,3,3,3,3,1], header=None)</span>

Did it work? Print out the data.  It should look like

<span style="color:red">df.head()</span>

#### Exercise 4.1:

What does <span style="color:red">df.shape</span> output?

#### Exercise 4.2:

Find out how to rename the DataFrame columns (which is labelled as 0, 1, 2, 3, 4, 5, 6, 7) at the moment.  Rename them to ‘ID', 'Gender', 'Visit', 'HR', 'SBP', 'DBP', 'DX', 'AE’.

If you inspect the whole DataFrame, there will be NaN in some of the fields.  NaN stands for “Not A Number”.

## Substituting Values

Now, let's have a closer look at the data. Print out the column "Gender":

<span style="color:red">df['Gender']</span>

We can see a missing value ('NaN'), some likely incorrect values ('<span style="color:red">X</span>', '<span style="color:red">2</span>'), and some inconsistency in the labelling, (both '<span style="color:red">f</span>' and '<span style="color:red">F</span>' used to denote female and both '<span style="color:red">m</span>' and '<span style="color:red">M</span>' used to denote male). We can select the rows containing lowercase '<span style="color:red">f</span>' as follows:

<span style="color:red">df.loc[df['Gender']=='f']</span>

We can fix the inconsistent cases by replacing the value of 'f' by 'F' in each of these rows:

<span style="color:red">df.loc[df['Gender']=='f','Gender'] = 'F'</span>

Print out the table to see if we've fixed the problem.

<span style="color:red">df['Gender']</span>

#### Exercise 4.3:

Do the same for 'm'.

You can also filter it to view entries that are not 'M' or 'F'.

<span style="color:red">df[(df['Gender'] != 'F') & (df['Gender'] != 'M')]</span>

#### Exercise 4.4:

What would you do with the '2', 'X' and 't'?  Discuss this with your coursemates.

## Imputing Data

We have substituted some data that is rather obvious, but there are some such as '2' for gender, of 'XX5' for ID, or '2999' for the year, or simply 'NaN'.  What are the techniques to rectify them or should they be rectified?  Some are also quite obvious, like '98' for the year, which we are very sure should be '1998'.

#### Exercise 4.5:

Substitute the '98' with '1998'.

For the missing values ('NaN'), depending on the situation, the common ways of handling them are

Deleting the entire entry (row)
Imputing with the mean, mode or median value
Impute with an interpolated value
For categorical data, impute with most likely value (most frequent)
However, from practical experience, deleting the entire entry is usually the most common if you do have a large dataset.

## Removing Entries

We can aggregate values over columns.  Let's say we want to know the average Systolic Blood Pressure (SBP) across the patients, we can use the mean function to aggregate the values:

<span style="color:red">df['SBP'].mean()</span>

And if we'd like to know the average blood pressure for males and females separately, all we need to do is first "groupby" the column "Gender":

<span style="color:red">df.groupby('Gender')['SBP'].mean()</span>

Note that we are seeing aggregate values also for the erroneous values '2' and 'X' (if you have not treated them in Exercise 4.4).

#### Exercise 4.6:

We might want to see average values for other variables as well, such as the Diastolic Blood Pressure (DBP). Write a <span style="color:red">groupby()</span> on the column “<span style="color:red">Gender</span>” for the <span style="color:red">mean</span>() values of SBP and DBP.  Hint: You can create a list for the SBP and DBP.

If we want to see the aggregate values for all other columns we can do that by simply not listing any columns:

<span style="color:red">df.groupby('Gender').mean()</span>

Note: The above code does not seem to run on MacOS (please correct us if this is incorrect).  Under this circumstance, you will need to specify the specific columns.

Looking back at the output. Imagine we wish to remove anomalously large values for SBP. From the table it looks like SBP is generally below 350. Let's remove values larger than that:

<span style="color:red">df = df[df['SBP'] <= 350]</span>

Print out the shape of the resulting table to find out how many rows are left:

<span style="color:red">df.shape</span>

Which row(s) were removed? Print out the table to see:

<span style="color:red">df</span>

Note that the original row numbers are still being used even though certain rows have been removed.  If you inspect it closely, you will notice that rows 7, 19, 20 and 28 are missing. For the Heart Rate (HR) column, let's assume we know that values ought to normally lie in the range 60 to 100 (https://www.ncbi.nlm.nih.gov/books/NBK493201/Links to an external site.). We could remove any values below 60, but that might remove some low but possibly correct values (as the lowest known heart rate is 26).

#### Exercise 4.7: 

Let's remove patients with a heart rate lower than 30 to be safe

## Duplicated Values

From the DataFrame, you may have noticed that there are also duplicated values.  You can use the duplicate() function in pandas to discover whether there are any. 

<span style="color:red">duplicate_values = df.duplicated()  
df[duplicate_values]</span>

You will notice that it returns nothing as it tries to find exact match for the whole row.  You may want to narrow it down to maybe finding duplicated 'ID', and decide which row you would like to keep.  It can get complicated as you may want to keep the row with the latest date and so on.  However, for brevity, the following code will identify the duplicates but not the first occurence.

<span style="color:red">duplicate_values = df['ID'].duplicated()  
df[duplicate_values]</span>

You should find that there may be 2 entries that have duplicated 'ID'.  You can decide how to would want to handle it.

## Stanford Data Wrangler

This section is to allow you to explore another wrangling tool that has been successfully developed into a commercial product called Trifacta, which was then bought over by Alteryx (https://www.alteryx.com/Links to an external site.).  This was originally jointly developed at Stanford/Berkeley and you can still access a demo version of it at http://vis.stanford.edu/wrangler/Links to an external site. . Do watch the short demo video and then explore the tool online using the 3 examples they provided.  For a start, you can try the example data (“Crime”) by clicking on the “Wrangle” button on the top right.

Upon completion of the exercise (on your own), try to copy and paste the <span style="color:red">patients.txt</span> date into it and try it out (note that this is not that important but it is something for you to experience).  (For the Malaysia campus, please ask your lecturer on possible access to Alteryx).

## My code part

#### Exercise 4.1:

In [2]:
import pandas as pd  
df = pd.read_fwf('data/patients.txt', colspecs='infer', header=None)

In [3]:
print(df)

                          0   1
0   001M11/11/1998 88140 80  10
1   016F11/13/1998 84120 78  X0
2   033X10/21/1998 68190100  31
3   004F01/01/1999101200120  5A
4   XX5M05/07/1998 68120 80  10
5   006 06/15/1999 72102 68  61
6   007M08/32/1998 88148102   0
7         008F08/08/1998210  70
8   009M09/25/1999 86240180  41
9      010F10/19/1999 40120  10
10  011M13/13/1998 68300 20  41
11  012M10/12/98   60122 74   0
12  013208/23/1999 74108 64   1
13  014M02/02/1999 22130 90   1
14  002f11/13/1998 84120 78  X0
15  003M11/12/1999 58112 74   0
16  015F           82148 88  31
17     017F04/05/1999208 84  20
18  019M06/07/1999 58118 70   0
19        123M15/12/1999 60  10
20  321f          900400200  51
21   020F99/99/9999 10 20 8   0
22  022M10/10/1999 48114 82  21
23  023F12/31/1998 22 34 78   0
24  024F11/09/199876 120 80  10
25  025M01/01/1999 74102 68  51
26  027FNOTAVAIL  NA 166106  70
27  019M06/07/1999 58121 20  10
28  028F03/28/1998 66150 90  30
29           029M05/15/1998  41
30  006F

In [4]:
df.shape

(35, 2)

#### Exercise 4.2:

In [5]:
# Rename columns
df.columns = ['ID', 'Gender', 'Visit', 'HR', 'SBP', 'DBP', 'DX', 'AE']

# Display the first few rows to confirm changes
print(df.head())

ValueError: Length mismatch: Expected axis has 2 elements, new values have 8 elements

In [ ]:
# Define column widths manually
colspecs = [(0, 3), (3, 4), (4, 14), (14, 19), (19, 22), (22, 25), (25, 28), (28, 30)]

# Read the file with fixed column widths
df = pd.read_fwf('data/patients.txt', colspecs=colspecs, header=None)

# Rename columns
df.columns = ['ID', 'Gender', 'DOB', 'HR', 'SBP', 'DBP', 'DX', 'AE']

# Display the first few rows
print(df.head())


    ID Gender         DOB     HR  SBP  DBP  DX  AE
0  001      M  11/11/1998   8814  0 8  0.0  10 NaN
1  016      F  11/13/1998   8412  0 7  8.0  X0 NaN
2  033      X  10/21/1998   6819  010  0.0  31 NaN
3  004      F  01/01/1999  10120  012  0.0  5A NaN
4  XX5      M  05/07/1998   6812  0 8  0.0  10 NaN


In [7]:
df['Gender']

0       M
1       F
2       X
3       F
4       M
5     NaN
6       M
7       F
8       M
9       F
10      M
11      M
12      2
13      M
14      f
15      M
16      F
17      F
18      M
19      M
20      f
21      F
22      M
23      F
24      F
25      M
26      F
27      M
28      F
29      M
30      F
31      m
32      M
33      f
34      t
Name: Gender, dtype: object

In [8]:
df.loc[df['Gender']=='f','Gender'] = 'F'
df['Gender']

0       M
1       F
2       X
3       F
4       M
5     NaN
6       M
7       F
8       M
9       F
10      M
11      M
12      2
13      M
14      F
15      M
16      F
17      F
18      M
19      M
20      F
21      F
22      M
23      F
24      F
25      M
26      F
27      M
28      F
29      M
30      F
31      m
32      M
33      F
34      t
Name: Gender, dtype: object

#### Exercise 4.3:

In [10]:
# Fix lowercase 'm' -> 'M'
df.loc[df['Gender'] == 'm', 'Gender'] = 'M'
df['Gender']

0       M
1       F
2       X
3       F
4       M
5     NaN
6       M
7       F
8       M
9       F
10      M
11      M
12      2
13      M
14      F
15      M
16      F
17      F
18      M
19      M
20      F
21      F
22      M
23      F
24      F
25      M
26      F
27      M
28      F
29      M
30      F
31      M
32      M
33      F
34      t
Name: Gender, dtype: object

In [11]:
df[(df['Gender'] != 'F') & (df['Gender'] != 'M')]

,ID,Gender,DOB,HR,SBP,DBP,DX,AE
2,033,X,10/21/1998,6819,010,0.0,31,NaN
5,006,NaN,06/15/1999,7210,2 6,8.0,61,NaN
12,013,2,08/23/1999,7410,8 6,4.0,1,NaN
34,999,t,99/00/9999,10 2,0 8,NaN,0,NaN


#### Exercise 4.4:

#### Exercise 4.5:

In [13]:
df['DOB'] = df['DOB'].str.replace(r'\b98\b', '1998', regex=True)

df['DOB']


0     11/11/1998
1     11/13/1998
2     10/21/1998
3     01/01/1999
4     05/07/1998
5     06/15/1999
6     08/32/1998
7     08/08/1998
8     09/25/1999
9     10/19/1999
10    13/13/1998
11    10/12/1998
12    08/23/1999
13    02/02/1999
14    11/13/1998
15    11/12/1999
16           NaN
17    04/05/1999
18    06/07/1999
19    15/12/1999
20           NaN
21    99/99/9999
22    10/10/1999
23    12/31/1998
24    11/09/1998
25    01/01/1999
26      NOTAVAIL
27    06/07/1999
28    03/28/1998
29    05/15/1998
30    07/07/1999
31    31/05/2999
32     15/12/199
33           NaN
34    99/00/9999
Name: DOB, dtype: object

In [16]:
df['SBP'] = pd.to_numeric(df['SBP'], errors='coerce')
print(df['SBP'].mean())


150.8


In [17]:
df.groupby('Gender')['SBP'].mean()

Gender
2           NaN
F    111.000000
M    277.333333
X     10.000000
t           NaN
Name: SBP, dtype: float64

In [19]:
df[['SBP', 'DBP']] = df[['SBP', 'DBP']].apply(pd.to_numeric, errors='coerce')

df.groupby('Gender')[['SBP', 'DBP']].mean()


,SBP,DBP
Gender,,
2,NaN,4.000000
F,111.000000,3.818182
M,277.333333,1.666667
X,10.000000,0.000000
t,NaN,NaN


In [20]:
df.groupby('Gender').mean()

TypeError: agg function failed [how->mean,dtype->object]

In [22]:
df = df[df['SBP'] <= 350]
df.shape

(8, 8)